In [17]:
import geemap
import ee

ee.Authenticate()
ee.Initialize()
australia = geemap.Map(center=(-26.9, 124.24), zoom=4.7)
australia


Successfully saved authorization token.


Map(center=[-26.9, 124.24], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

In [18]:
ch4 = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CH4') .select('CH4_column_volume_mixing_ratio_dry_air') .filterDate('2019-06-01', '2019-07-16')

viz = {
  'min': 1750,
  'max': 1850,
  'opacity': 0.5,
  'palette': ['black']
};

australia.addLayer(ch4.mean(), viz, 'S5P CH4')

In [19]:
mean_methane = ch4.mean()
median_methane = ch4.median()
highest_methane = ch4.max()

# Display the calculated statistics
print("Mean Methane Concentration:", mean_methane.getInfo)
print("Median Methane Concentration:", median_methane.getInfo())

print("Highest Methane Concentration:", highest_methane.getInfo())


Mean Methane Concentration: <bound method Image.getInfo of <ee.image.Image object at 0x11f8482e0>>
Median Methane Concentration: {'type': 'Image', 'bands': [{'id': 'CH4_column_volume_mixing_ratio_dry_air', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Mode Methane Concentration: {'type': 'Image', 'bands': [{'id': 'CH4_column_volume_mixing_ratio_dry_air_mode', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Highest Methane Concentration: {'type': 'Image', 'bands': [{'id': 'CH4_column_volume_mixing_ratio_dry_air', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [20]:
buffer_size = 20000
kernel = ee.Kernel.circle(radius=buffer_size, units='meters')
mean_within_buffer = ch4.mean().reduceNeighborhood(
    reducer=ee.Reducer.mean(),
    kernel=kernel
)
threshold = 1850
anomaly_image = mean_within_buffer.gt(threshold)
australia.addLayer(anomaly_image.updateMask(anomaly_image), {'palette': 'red'}, 'Methane Anomalies')
australia.centerObject(anomaly_image)
australia

Map(bottom=5032.0, center=[-26.9, 124.24], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…